In [1]:
import joblib
import tempfile
import os
import pandas as pd
bid_number = 'P260030994'
outfile = os.path.join(tempfile.gettempdir(),'%s.data'%bid_number)
df_tp20_bid,df_tp20_bid_shpr,df_tp20_svc_grp,df_tp20_ceiling_svc,df_tp20_shpr_svc,df_ttpsvgp,df_zone_weight,df_tp_accessorial = joblib.load(outfile)

In [2]:
import numpy as np

1. create lead account shipper service

In [3]:
df_tp20_shpr_svc.head()

,NVP_BID_NR,SVC_GRP_NR,SVC_GRP_SUF_NR,SHR_AC_NR,CUS_AC_STT_DT,CNY_CD,CCY_CD,SHR_PJT_GRS_RPP_A,SHR_PJT_WVL_QY,SHR_PRR_GRS_RVN_A,SHR_PRR_NET_RVN_A,TPT_FRT_CLS_NR,CPE_CRR_NTL_NCV_PR,CPE_CRR_REG_NCV_PR,CPE_NTL_NRV_MIA_A,CPE_REG_NRV_MIA_A,RA_TRI_NR
0,P260030994,AHTSVBL,0,00004F93W3,2017-08-08,US,USD,31.7082,0.100,0.000,0.000,None,None,None,None,None,6
1,P260030994,AHTSVBM,0,00004F93W3,2017-08-08,US,USD,97.5446,1.394,779.815,444.500,None,None,None,None,None,6
2,P260030994,AHTSVBN,0,00004F93W3,2017-08-08,US,USD,30.6538,0.100,0.000,0.000,None,None,None,None,None,6
3,P260030994,AHTSVBO,0,00004F93W3,2017-08-08,US,USD,133.2249,1.505,690.920,393.830,None,None,None,None,None,6
4,P260030994,AHTSVBQ,0,00004F93W3,2017-08-08,US,USD,16.6467,0.100,27.235,16.885,None,None,None,None,None,6


In [4]:
tp20_shpr_svc_leadaccount = df_tp20_shpr_svc
tp20_shpr_svc_leadaccount['List_Rev'] = tp20_shpr_svc_leadaccount['SHR_PJT_WVL_QY'] * tp20_shpr_svc_leadaccount[
    'SHR_PJT_GRS_RPP_A']
tp20_shpr_svc_leadaccount = tp20_shpr_svc_leadaccount.groupby(by=['NVP_BID_NR', 'SHR_AC_NR']).agg(
    {'List_Rev': np.sum})

tp20_shpr_svc_leadaccount.head()

,,List_Rev
NVP_BID_NR,SHR_AC_NR,
P260030994,00004F93W3,1649.268053


In [5]:
tp20_shpr_svc_leadaccount = tp20_shpr_svc_leadaccount.reset_index()
tp20_shpr_svc_leadaccount = tp20_shpr_svc_leadaccount.sort_values(by=['NVP_BID_NR', 'List_Rev'], ascending=[1, 0])
tp20_shpr_svc_leadaccount = tp20_shpr_svc_leadaccount[['NVP_BID_NR', 'SHR_AC_NR']]
tp20_shpr_svc_leadaccount.head()

,NVP_BID_NR,SHR_AC_NR
0,P260030994,00004F93W3


2. create prior revenue shipper service

In [6]:
tp20_shpr_svc_priorrev = df_tp20_shpr_svc
tp20_shpr_svc_priorrev['Prior_GRS_Rev'] = tp20_shpr_svc_priorrev['SHR_PJT_WVL_QY'] * tp20_shpr_svc_priorrev['SHR_PRR_GRS_RVN_A']
tp20_shpr_svc_priorrev['Prior_NET_Rev'] = tp20_shpr_svc_priorrev['SHR_PJT_WVL_QY'] * tp20_shpr_svc_priorrev['SHR_PRR_NET_RVN_A']
tp20_shpr_svc_priorrev = tp20_shpr_svc_priorrev.groupby(by=['NVP_BID_NR', 'SVC_GRP_NR']).agg(
    {'Prior_GRS_Rev': np.sum, 'Prior_NET_Rev': np.sum})
tp20_shpr_svc_priorrev.head()

Prior_GRS_Rev  Prior_NET_Rev
NVP_BID_NR SVC_GRP_NR                              
P260030994 AHTSVBL           0.00000        0.00000
           AHTSVBM        1087.06211      619.63300
           AHTSVBN           0.00000        0.00000
           AHTSVBO        1039.83460      592.71415
           AHTSVBQ           2.72350        1.68850

In [7]:
tp20_shpr_svc_priorrev = tp20_shpr_svc_priorrev.reset_index()
tp20_shpr_svc_priorrev = tp20_shpr_svc_priorrev[['NVP_BID_NR','SVC_GRP_NR','Prior_GRS_Rev','Prior_NET_Rev']]
tp20_shpr_svc_priorrev.head()

,NVP_BID_NR,SVC_GRP_NR,Prior_GRS_Rev,Prior_NET_Rev
0,P260030994,AHTSVBL,0.00000,0.00000
1,P260030994,AHTSVBM,1087.06211,619.63300
2,P260030994,AHTSVBN,0.00000,0.00000
3,P260030994,AHTSVBO,1039.83460,592.71415
4,P260030994,AHTSVBQ,2.72350,1.68850


3. Shipper Info

In [8]:
Region_array = df_tp20_bid_shpr[['NVP_BID_NR', 'REG_NR']].groupby(['NVP_BID_NR']).apply(
    lambda g: g.groupby('REG_NR').count().idxmax())
Region_array.columns = ['Region']
District_array = df_tp20_bid_shpr[['NVP_BID_NR', 'DIS_NR']].groupby(['NVP_BID_NR']).apply(
    lambda g: g.groupby('DIS_NR').count().idxmax())
District_array.columns = ['District']
LeadAccount_array = tp20_shpr_svc_leadaccount[['NVP_BID_NR', 'SHR_AC_NR']].groupby(['NVP_BID_NR']).apply(
    lambda g: g.groupby('SHR_AC_NR').count().idxmax())
LeadAccount_array.columns = ['LeadAccount']

In [9]:
Region_array.head()

,Region
NVP_BID_NR,
P260030994,7


In [10]:
District_array.head()

,District
NVP_BID_NR,
P260030994,36


In [11]:
LeadAccount_array.head()

,LeadAccount
NVP_BID_NR,
P260030994,00004F93W3


In [12]:
# Step 3.2 : Merge Bid info & Shipper info
tp20_bid_shpr_Merged = df_tp20_bid.merge(Region_array, left_on='NVP_BID_NR', right_index=True).merge(
    District_array, left_on='NVP_BID_NR', right_index=True).merge(
    LeadAccount_array, left_on='NVP_BID_NR', right_index=True)
tp20_bid_shpr_Merged

,NVP_BID_NR,NVP_BID_CGY_TYP_CD,NVP_BID_STS_CD,BID_STS_EFF_DT,NVP_BID_NA,NVP_BID_INI_DT,NVP_BID_EFF_DT,NVP_BID_END_DT,EMP_NA,CPE_ETM_OR_IR,NVP_BID_PRP_DT,NVP_BID_RQT_RSN_CD,Region,District,LeadAccount
0,P260030994,2,P,2017-02-01,,2017-02-01,2017-02-11,2046-12-29,,N,1999-12-12,2,7,36,00004F93W3


# TP20_BID_TABLE

In [13]:
#ensure category is a string
tp20_bid_shpr_Merged['NVP_BID_CGY_TYP_CD'] = '0' + tp20_bid_shpr_Merged['NVP_BID_CGY_TYP_CD'].astype(str)

# Step 3.3 : Recreate TP Bid Table
tp_bid_table = pd.DataFrame(
    {'BidNumber': tp20_bid_shpr_Merged['NVP_BID_NR'],
     'Status': tp20_bid_shpr_Merged['NVP_BID_STS_CD'],
     'StatusDate': tp20_bid_shpr_Merged['BID_STS_EFF_DT'],
     'ProposedDate': tp20_bid_shpr_Merged['NVP_BID_PRP_DT'],
     'InitiationDate': tp20_bid_shpr_Merged['NVP_BID_INI_DT'],
     'Category': np.where(tp20_bid_shpr_Merged['NVP_BID_CGY_TYP_CD'] == '00', "Not known",
                          np.where(tp20_bid_shpr_Merged['NVP_BID_CGY_TYP_CD'] == '01', "National",
                                   np.where(tp20_bid_shpr_Merged['NVP_BID_CGY_TYP_CD'] == '02', "Major",
                                            np.where(tp20_bid_shpr_Merged['NVP_BID_CGY_TYP_CD'] == '03', "Key",
                                                     np.where(
                                                         tp20_bid_shpr_Merged['NVP_BID_CGY_TYP_CD'] == '04',
                                                         "Small", "Not known"))))),
     'Region': tp20_bid_shpr_Merged['Region'],
     'District': tp20_bid_shpr_Merged['District'],
     'LeadAccount': tp20_bid_shpr_Merged['LeadAccount']})
tp_bid_table

,BidNumber,Category,District,InitiationDate,LeadAccount,ProposedDate,Region,Status,StatusDate
0,P260030994,Major,36,2017-02-01,00004F93W3,1999-12-12,7,P,2017-02-01


In [14]:
# Step 4.1: Merge IAS SVC and ttpsvgp
# Merge variables to ISA SVC tables

df_ttpsvgp.drop('PND_STS_CD', 1, inplace=True)
max_factor = df_ttpsvgp['TRG_PSE_FCR_NR'].max()
df_ttpsvgp = df_ttpsvgp[df_ttpsvgp.TRG_PSE_FCR_NR == max_factor]
df_ttpsvgp = df_ttpsvgp.groupby(['NVP_BID_NR', 'SVC_GRP_NR']).agg('max')
df_ttpsvgp = df_ttpsvgp.reset_index()

tp20_svc_grp = df_tp20_svc_grp.merge(df_ttpsvgp, how='left')

## Merge Prior Net/Gross Rev
tp20_svc_grp = tp20_svc_grp.merge(tp20_shpr_svc_priorrev, how='left')


In [15]:
tp20_svc_grp

,SVC_GRP_NR,SVC_GRP_SUF_NR,NVP_BID_NR,TRG_PSE_SVC_ID,SVC_FEA_TYP_CD,SVC_FEA_TYP_DSC_TE,PKG_CHA_TYP_CD,PKG_CHA_TYP_DSC_TE,SVC_TYP_CD,SVC_TYP_CD_DSC_TE,...,CU_SMP_AVG_WGT_QY,CU_SMP_AVG_NR,CALC_CUBE_PACKAGE_DENSITY,CPE_ETM_RPP_A,SVC_GRP_TRG_PSE_A,SVC_TRG_LOW_RNG_A,SVC_TRG_HI_RNG_A,TRG_PSE_FCR_NR,Prior_GRS_Rev,Prior_NET_Rev
0,AHTSVBL,0,P260030994,N1DALTRSCX,SCX,,LTR,,1DA,,...,0.000000,0.035300,0.0000,18.0207,18.434,20.420,18.276,0226,0.000000,0.00000
1,AHTSVBM,0,P260030994,N1DAPKGSCX,SCX,,PKG,,1DA,,...,18.404256,1.369229,13.4413,45.1006,45.583,51.460,45.116,0226,1087.062110,619.63300
2,AHTSVBN,0,P260030994,N1DPLTRSCX,SCX,,LTR,,1DP,,...,0.000000,0.035300,0.0000,17.1626,17.761,19.689,17.608,0226,0.000000,0.00000
3,AHTSVBO,0,P260030994,N1DPPKGSCX,SCX,,PKG,,1DP,,...,19.875000,1.579063,12.5866,64.3729,65.381,73.635,64.726,0226,1039.834600,592.71415
4,AHTSVBQ,0,P260030994,N2DALTRSCX,SCX,,LTR,,2DA,,...,0.000000,0.035300,0.0000,9.0336,9.189,10.236,9.106,0226,2.723500,1.68850
5,AHTSVBR,0,P260030994,N2DAPKGSCX,SCX,,PKG,,2DA,,...,26.760870,2.053847,13.0296,77.0795,78.373,87.327,77.663,0226,174.712000,108.32000
6,AHTSVBS,0,P260030994,N2DMLTRSPB,SPB,,LTR,,2DM,,...,0.000000,0.035300,0.0000,13.6386,14.089,15.520,13.975,0226,0.000000,0.00000
7,AHTSVBT,0,P260030994,N2DMPKGSPB,SPB,,PKG,,2DM,,...,0.000000,0.155893,0.0000,13.6386,14.134,15.632,14.015,0226,0.000000,0.00000
8,AHTSVBU,0,P260030994,N3DSPKGSCX,SCX,,PKG,,3DS,,...,12.750000,0.978412,13.0313,31.3165,31.931,35.334,31.661,0226,43.267000,29.42000
9,AHTSVDZ,0,P260030994,NGNDPKGSPB,SPB,,PKG,,GND,,...,20.620000,1.490597,13.8334,9.7644,9.776,10.861,9.702,0226,350936.651070,278117.86857


In [16]:

# Step 4.2: Recreate TP SVC table
# Create FedEx competitor identifier
Competitor_Cate = {15: 'FedEx',
                   16: 'FedEx',
                   60: 'FedEx',
                   83: 'FedEx'}

In [17]:
# Create SVC table
tp_bid_svc_table = pd.DataFrame(
    {'BidNumber': tp20_svc_grp['NVP_BID_NR'],
     'SVC_GRP_NR': tp20_svc_grp['SVC_GRP_NR'],
     'MVM_DRC_CD': tp20_svc_grp['MVM_DRC_CD'],
     'SVM_TYP_CD': tp20_svc_grp['SVC_TYP_CD'],
     'PKG_CHA_TYP_CD': tp20_svc_grp['PKG_CHA_TYP_CD'],
     'SVC_FEA_TYP_CD': tp20_svc_grp['SVC_FEA_TYP_CD'],
     'True_Density': tp20_svc_grp['CALC_CUBE_PACKAGE_DENSITY'],
     'Competitor': tp20_svc_grp['CPE_CRR_NR'].map(Competitor_Cate).fillna('Other'),
     # Need engineer, a mapping table from UPS?
     'CPE_CRR_NR': tp20_svc_grp['CPE_CRR_NR'],
     'Volume': tp20_svc_grp['BID_UPS_PJT_WVL_QY'],
     'Prior_Volume': tp20_svc_grp['BID_CUS_PRR_WVL_QY'],
     'List_Rev_PP': tp20_svc_grp['NVP_BID_FRT_GRR_A'],
     'List_Rev_PP_Freight': tp20_svc_grp['UPS_PRJ_GRS_RPP_A'],
     'Marginal_Cost_PP': tp20_svc_grp['BID_MRG_CPP_A'],
     'Fully_Allocated_Cost_PP': tp20_svc_grp['BID_FA_CPP_A'],
     'Comp_Keyed_Rev_PP': tp20_svc_grp['CPE_SVC_RPP_A'],
     'Comp_Est_Rev_PP': tp20_svc_grp['CPE_ETM_RPP_A'],
     'Comp_Net_Rev_PP': np.where(tp20_svc_grp['CPE_SVC_RPP_A'] == 0, tp20_svc_grp['CPE_ETM_RPP_A'],
                                 tp20_svc_grp['CPE_SVC_RPP_A']),
     'PriorGrossRevenue': tp20_svc_grp['Prior_GRS_Rev'] / 13,
     'PriorNetRevenue': tp20_svc_grp['Prior_NET_Rev'] / 13,
     'Target_Nominal_Rev_PP': tp20_svc_grp['SVC_GRP_TRG_PSE_A'],
     'Target_Low_Rev_PP': tp20_svc_grp['SVC_TRG_LOW_RNG_A'],
     'Target_High_Rev_PP': tp20_svc_grp['SVC_TRG_HI_RNG_A']
     })

tp20_shpr_svc = df_tp20_shpr_svc[['NVP_BID_NR','SVC_GRP_NR','RA_TRI_NR']]
tp_bid_svc_table = tp_bid_svc_table.merge(tp20_shpr_svc, how="left")
tp_bid_svc_table = tp_bid_svc_table.rename(columns={'RA_TRI_NR':'Billing_Tier'})


# Transformed Data

In [18]:
tp_bid_table

,BidNumber,Category,District,InitiationDate,LeadAccount,ProposedDate,Region,Status,StatusDate
0,P260030994,Major,36,2017-02-01,00004F93W3,1999-12-12,7,P,2017-02-01


In [19]:
tp_bid_svc_table

,BidNumber,CPE_CRR_NR,Comp_Est_Rev_PP,Comp_Keyed_Rev_PP,Comp_Net_Rev_PP,Competitor,Fully_Allocated_Cost_PP,List_Rev_PP,List_Rev_PP_Freight,MVM_DRC_CD,...,SVC_FEA_TYP_CD,SVC_GRP_NR,SVM_TYP_CD,Target_High_Rev_PP,Target_Low_Rev_PP,Target_Nominal_Rev_PP,True_Density,Volume,NVP_BID_NR,Billing_Tier
0,P260030994,000016,18.0207,0,18.0207,Other,7.686662,31.7897,31.5815,N,...,SCX,AHTSVBL,1DA,18.276,20.420,18.434,0.0000,0.0001,P260030994,6
1,P260030994,000016,45.1006,0,45.1006,Other,18.476748,98.0566,93.4621,N,...,SCX,AHTSVBM,1DA,45.116,51.460,45.583,13.4413,1.4364,P260030994,6
2,P260030994,000016,17.1626,0,17.1626,Other,9.113105,30.6538,30.6538,N,...,SCX,AHTSVBN,1DP,17.608,19.689,17.761,0.0000,0.0001,P260030994,6
3,P260030994,000016,64.3729,0,64.3729,Other,26.100491,134.1009,131.2690,N,...,SCX,AHTSVBO,1DP,64.726,73.635,65.381,12.5866,0.8359,P260030994,6
4,P260030994,000016,9.0336,0,9.0336,Other,3.595988,16.6467,16.6467,N,...,SCX,AHTSVBQ,2DA,9.106,10.236,9.189,0.0000,0.0779,P260030994,6
5,P260030994,000016,77.0795,0,77.0795,Other,27.910374,144.6049,142.3900,N,...,SCX,AHTSVBR,2DA,77.663,87.327,78.373,13.0296,1.9475,P260030994,6
6,P260030994,000016,13.6386,0,13.6386,Other,7.366513,22.9785,22.7682,N,...,SPB,AHTSVBS,2DM,13.975,15.520,14.089,0.0000,0.0001,P260030994,6
7,P260030994,000016,13.6386,0,13.6386,Other,7.911627,24.1502,23.8170,N,...,SPB,AHTSVBT,2DM,14.015,15.632,14.134,0.0000,0.0001,P260030994,6
8,P260030994,000016,31.3165,0,31.3165,Other,13.014709,54.9695,54.1154,N,...,SCX,AHTSVBU,3DS,31.661,35.334,31.931,13.0313,1.0898,P260030994,6
9,P260030994,000016,9.7644,0,9.7644,Other,8.986472,17.8162,14.7302,N,...,SPB,AHTSVDZ,GND,9.702,10.861,9.776,13.8334,53.8723,P260030994,6


In [20]:
tp_bid_svc_table.columns

Index([u'BidNumber', u'CPE_CRR_NR', u'Comp_Est_Rev_PP', u'Comp_Keyed_Rev_PP',
       u'Comp_Net_Rev_PP', u'Competitor', u'Fully_Allocated_Cost_PP',
       u'List_Rev_PP', u'List_Rev_PP_Freight', u'MVM_DRC_CD',
       u'Marginal_Cost_PP', u'PKG_CHA_TYP_CD', u'PriorGrossRevenue',
       u'PriorNetRevenue', u'Prior_Volume', u'SVC_FEA_TYP_CD', u'SVC_GRP_NR',
       u'SVM_TYP_CD', u'Target_High_Rev_PP', u'Target_Low_Rev_PP',
       u'Target_Nominal_Rev_PP', u'True_Density', u'Volume', u'NVP_BID_NR',
       u'Billing_Tier'],
      dtype='object')